In [1]:
import pandas as pd
import sqlite3
pd.set_option('display.max_colwidth', None)

# Create Database

In [2]:
conn = sqlite3.connect("./dannys_dinner.db")
cursor = conn.cursor()

In [3]:
cursor.executescript('''
  CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');


CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');


CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
''')

In [4]:
conn.commit()

Verify that database is created successfully

In [17]:
# test
cursor.execute('''SELECT * FROM members''').fetchall()

[('A', '2021-01-07'), ('B', '2021-01-09')]

# Do the challenge
https://8weeksqlchallenge.com/case-study-1/

In [6]:
# get all tables from the database
cursor.execute('''SELECT name FROM sqlite_master WHERE type="table"''').fetchall()

[('sales',), ('menu',), ('members',)]

Question 1: What is the total amount each customer spent at the restaurant?

In [118]:
pd.read_sql_query('''
SELECT customer_id, SUM(price) as total_amount
FROM sales
INNER JOIN menu ON menu.product_id = sales.product_id
GROUP BY customer_id
''',
                  conn)

,customer_id,total_amount
0,A,76
1,B,74
2,C,36


Question 2: How many days has each customer visited the restaurant?

In [119]:
pd.read_sql_query('''
SELECT customer_id, COUNT(distinct order_date) as total_visits
FROM sales
GROUP BY customer_id
'''
, conn)

,customer_id,total_visits
0,A,4
1,B,6
2,C,2


Question 3: What was the first item from the menu purchased by each customer?

In [121]:
pd.read_sql_query('''
WITH temp AS (
  SELECT customer_id, order_date, product_name,
  DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date) as date_rank
  FROM sales
  JOIN menu ON menu.product_id = sales.product_id
)

SELECT  distinct customer_id, product_name
FROM temp
WHERE date_rank=1
'''
, conn)

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry
3,C,ramen


In [31]:
# Another way to do the query
pd.read_sql_query('''
SELECT
  s.customer_id,
  s.order_date,
  m.product_name
FROM sales s
JOIN menu m ON s.product_id = m.product_id
WHERE s.order_date = (
  SELECT MIN(order_date)
  FROM sales
  WHERE customer_id = s.customer_id
)
'''
, conn)



,customer_id,order_date,product_name
0,A,2021-01-01,sushi
1,A,2021-01-01,curry
2,B,2021-01-01,curry
3,C,2021-01-01,ramen
4,C,2021-01-01,ramen


Question 5: What is the most purchased item on the menu and how many times was it purchased by all customers?


In [125]:
pd.read_sql_query('''
SELECT product_name, COUNT(product_name) as buying_times
FROM sales s
JOIN menu m ON s.product_id = m.product_id
GROUP BY product_name
ORDER BY buying_times DESC
LIMIT 1
''',conn)

,product_name,buying_times
0,ramen,8


Question 6: Which item was the most popular for each customer?


In [133]:
pd.read_sql_query('''
WITH temp AS (
  SELECT
    customer_id, product_name, COUNT(product_name) as buying_times,
    RANK() OVER (PARTITION BY s.customer_id ORDER BY count(s.product_id) DESC) as rank
  FROM sales s
  JOIN menu m ON s.product_id = m.product_id
  GROUP BY customer_id, product_name
  ORDER BY customer_id, buying_times DESC)

SELECT customer_id, product_name
FROM temp
WHERE rank=1

''', conn)

,customer_id,product_name
0,A,ramen
1,B,sushi
2,B,ramen
3,B,curry
4,C,ramen


Question 7: Which item was purchased first by the customer after they became a member?


In [68]:
pd.read_sql_query('''
WITH temp AS (SELECT
  sales.customer_id, sales.order_date, menu.product_name, members.join_date,
  RANK() OVER (PARTITION BY sales.customer_id ORDER BY sales.order_date) AS rank
FROM sales
JOIN menu  ON sales.product_id = menu.product_id
JOIN members ON members.customer_id = sales.customer_id
WHERE join_date < order_date)

SELECT  customer_id, product_name
FROM temp
WHERE rank=1
''', conn)

,customer_id,product_name
0,A,ramen
1,B,sushi


Question 8: Which item was purchased just before the customer became a member?


In [77]:
pd.read_sql_query('''
 WITH temp AS (SELECT
  sales.customer_id, sales.order_date, menu.product_name, members.join_date,
  RANK() OVER (PARTITION BY sales.customer_id ORDER BY sales.order_date DESC) AS rank
FROM sales
JOIN menu  ON sales.product_id = menu.product_id
JOIN members ON members.customer_id = sales.customer_id
WHERE join_date > order_date )

SELECT customer_id, product_name
FROM temp
WHERE rank=1
''',
                  conn)

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,sushi


Question 9: What is the total items and amount spent for each member before they became a member?


In [87]:
pd.read_sql_query('''
 SELECT
  sales.customer_id,
  COUNT(product_name) AS total_items,
  SUM(menu.price) AS amount_spend
FROM sales
JOIN menu  ON sales.product_id = menu.product_id
JOIN members ON members.customer_id = sales.customer_id  AND members.join_date > order_date
GROUP BY sales.customer_id
''',
                  conn)

,customer_id,total_items,amount_spend
0,A,2,25
1,B,3,40


Question 10: If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?


In [97]:
pd.read_sql_query('''
WITH temp AS (SELECT sales.customer_id,
      menu.product_name,
      CASE
        WHEN product_name='sushi' THEN 20*price
        ELSE price*10
      END AS price
FROM sales
JOIN menu ON menu.product_id = sales.product_id
)

SELECT customer_id, SUM(price) AS total_points
FROM temp
GROUP BY customer_id
''',
                  conn)

,customer_id,total_points
0,A,860
1,B,940
2,C,360


Question 11: In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [155]:
pd.read_sql_query('''
WITH temp AS (SELECT
      sales.customer_id,
      menu.product_name,
      members.join_date,
      sales.order_date,
      strftime('%m', order_date) AS month,
      strftime('%Y', order_date) AS year,
      CASE
        WHEN DATE(sales.order_date) BETWEEN DATE(members.join_date) AND DATE(members.join_date, '+6 days') THEN 20*price
        WHEN product_name='sushi' THEN 20*price
        ELSE  price*10
      END AS points
FROM sales
JOIN menu ON menu.product_id = sales.product_id
JOIN members ON members.customer_id = sales.customer_id AND month='01'
)

SELECT customer_id, SUM(points) AS total_points
FROM temp
GROUP BY customer_id


''',
                  conn)

,customer_id,total_points
0,A,1370
1,B,820


Close the database connection

In [ ]:
cursor.close()